# ASDI Demo Cases - Using TAP and a MAST API

This notebook shows how to use a TAP service to download pristine synthetic data from the MAST archive, and a second service that performs instrument simulations on that data server-side.  

For more information, see the ASDI Prototype documentation here: https://archive.stsci.edu/access-mast-data/asdi/asdi-prototype-demo

***
## Imports

In [ ]:
# Use the astroquery TapPlus library as our client to the data service.
from astroquery.utils.tap.core import TapPlus

# For handling ordinary astropy Tables in responses
from astropy.table import Table

# For displaying and manipulating some types of results
import astropy
import time
import numpy as np
import astropy.io.fits as fits
import requests
import os
from zipfile import ZipFile 

#plotting routines
import matplotlib
from matplotlib import pyplot

## TAP Service Introduction

Table Access Protocol (TAP) services allow more direct and flexible access to astronomical data than the simpler types of IVOA standard data services. Queries are built with the SQL-like Astronomical Data Query Language (ADQL), and can include geographic / spatial queries as well as filtering on other characteristics of the data. This also allows the user fine-grained control over the returned columns, unlike the fixed set of coumns returned from cone, image, and spectral services.

## Service Specific Configuration

Every TAP service has a "Base URL" plus associated endpoints for synchronous and asynchronous queries, as well as status and capability information, and sometimes service-provided sample queries. The endpoints are predefined in the TAP standard, so clients can infer them using the base. We therefore only have to provide astroquery that base.

In [ ]:
TAP_URL = "http://vaotest.stsci.edu/ASDI/tapservice.aspx"
TAP_service = TapPlus(url=TAP_URL)

### Browsing the Schema

TAP gives us access to descriptive metadata for this catalog. We can use this to narrow searches and filter our results. For the current catalog, there is only one table, with columns for the project name, nullable fields relevant to various projects, and a public path for downloading associated files.

In [ ]:
table_descriptions = TAP_service.load_tables()
print('\n')
for table in table_descriptions:
    if(not table.name.startswith('tap_schema')):
        print('TAP table: ' + table.name)
        print(table.description)
        print('\n')
        for i, column in enumerate(table.columns):
            print(column.name)
            print(column.description,'\n')

***
# ASDI CGM Use Case 1
Search for simulated CGM sightlines at 1.8 <= z <= 2.5 with O VI and Si II absorption. To compare with our own data, we want only sightlines with OVI column density > 10^12.75 cm-2. First we search for FOGGIE simulation lines of sight. We remotely search a catalog with redshifts and line column densities, impact parameters, galaxy stellar masses, galaxy star formation rates, and other quantities. 

In this example, we download the entire catalog for the resulting sightlines.  However, we can also filter server-side by replacing "SELECT \*" with, e.g., "SELECT lineName, extNum, redshift, totalColumn, impact, mstar, sfr, publicPath"

In [ ]:
# Launch TAP query
job = TAP_service.launch_job("""
            SELECT *
            FROM dbo.ASDISpectra1DCGM 
            WHERE projectName = 'foggie' AND
            (lineName = 'O VI 1032' OR lineName = 'Si II 1260') AND
            (redshift >= 1.8 and redshift <= 2.5) AND
            totalColumn >= 12.75
            """)
# Obtain results
foggie_results = job.get_results()

# Print available columns in resulting catalog
for c in foggie_results.columns: print(c)

In [ ]:
# Display example entires in resulting catalog
foggie_results[['lineName','redshift','totalColumn','impact','publicPath']][0:10]

***
# ASDI CGM Use Case 2

From the search results, access the pristine synthetic spectra at the intrinsic simulation resolution with no noise added. Then, call the ASDI data simulation service to generate custom mock spectra at the STIS/E140H resolution (artificially redshifting the line if necessary). Demonstrate spectra with S/N= 5, 25, 100 per resolution element.

In [ ]:
#choose a few random results to examine
np.random.seed(0)
interesting_los=foggie_results[np.random.randint(0,len(foggie_results),3)]

interesting_los

In [ ]:
#download a pristine simulated spectrum
example_file='http://archive.stsci.edu'+interesting_los['publicPath'][1]
print(example_file)

#create a local folder to save the file
save_location="ASDI_Sims"
if not os.path.lexists(save_location):
    os.mkdir(save_location)

r = requests.get(example_file, allow_redirects=True)
saved_file=os.path.join(save_location,os.path.basename(example_file))
open(saved_file, 'wb').write(r.content)
print('File downloaded: {} bytes'.format(r.headers['Content-length']))
hdulist=fits.open(saved_file)
print(hdulist.info())


In [ ]:
#plot pristine spectrum as well as a vector of intrinsic simulation data

#access pristine spectrum
this_hdu=hdulist[interesting_los[1]['extName']]
pristine_data=this_hdu.data

#access corresponding physical information (see headers for more information)
phys_hdu=hdulist['Physical']
phys_data=phys_hdu.data

fig, (ax1, ax2) = pyplot.subplots(1,2, figsize=(12, 6),dpi=100)
pyplot.subplots_adjust(hspace=0.2,wspace=0.2,top=0.88,bottom=0.12,left=0.06,right=0.99)
ax1.plot(pristine_data['disp_obs'],pristine_data['flux_obs'])

ax1.set_xlabel(r'$\lambda_{\rm obs} (\AA)$',size=25)
ax1.set_ylabel('flux',size=20)
ax1.annotate(this_hdu.header['EXTNAME'],(0.6,0.15),xycoords='axes fraction',size=25)
#ax1.annotate(r'$z_{\rm obs}$'+'={:4.2f}'.format(fo[3].header['OBS_Z']),(0.6,0.08),xycoords='axes fraction',size=25)
ax1.set_title('Pristine input spectrum',size=25)

ax2.semilogy(phys_data['los_position'],phys_data['metallicity'])
ax2.set_xlabel('Position along sightline (kpc)',size=25)
ax2.set_ylabel('Metallicity (Zsun)',size=20)

ax2.set_title('Physical Information',size=25)
fig.savefig(os.path.join(save_location,'asdi1.png'),dpi=300)

### Data Simulation

We can use this information to call the MAST service which applies instrument signatures to pristine data. This API works on one line at a time, and does the simulation server-side to be closer to the data and therefore faster than running locally. The service returns a FITS file, which we can inspect and open.

Pull out the first of our filtered rows, and call the service:

In [ ]:
example_row = interesting_los[0]

example_line_name = example_row['lineName']
example_ext_num = example_row['extNum']
example_path = example_row['publicPath']

STIS_sim_filename = os.path.basename(example_path).replace('pristine', 'stis')
print(STIS_sim_filename)

In [ ]:
inst_sim_url = "https://masttest.stsci.edu/asdi/api/v0.1/addsignature"

#construct a dictionary of input parameters
PARAMS = {'instrument':'STIS', #currently the only option for instrument
          'element1':'E140H', #currently the only option for instrument element
          'line_name': example_line_name, 
          'ext_num': example_ext_num, 
          'public_path': example_path,
          'snratioresel': 25,
          'override_observed_wavelength': 1500.0} 

#pass the service URL and parameters to Python requests call
r = requests.get(url = inst_sim_url, params = PARAMS, allow_redirects = True) 

#specify the FITS file in which to save the results
stis_file_path=os.path.join(save_location,STIS_sim_filename)

#response is returned as a JSON object which can be inspected for status and content.
if r.status_code != 200: #show what went wrong
    #print(r.headers)
    print(r.text)
else:
    #if successful, write the content to a FITS file
    open(stis_file_path, 'wb').write(r.content)

In [ ]:
fo=fits.open(stis_file_path)
fo.info()
print(fo['SyntheticData'].data.columns)

***
### Comparing Pristine and Instrument-Specific Data

Files in this public storage can be accessed via http, so we can download them using Python requests and compare:

In [ ]:
source_file = 'http://archive.stsci.edu'+example_row['publicPath']
print(source_file)

r = requests.get(source_file, allow_redirects=True)
open(os.path.join(save_location,os.path.basename(source_file)), 'wb').write(r.content)
print('File downloaded: {} bytes'.format(r.headers['Content-length']))

Now, display simulated data and compare to pristine input data.

In [ ]:
fig2, (ax1, ax2) = pyplot.subplots(1,2, figsize=(12, 6),dpi=100)
pyplot.subplots_adjust(hspace=0.2,wspace=0.2,top=0.88,bottom=0.12,left=0.06,right=0.99)

#plot STIS simulated data
simdata=fo['SyntheticData'].data
ax1.plot(simdata['lam_stis'],simdata['flux_stis'])

#plot original pristine spectrum with same x axis
ax2.plot(simdata['lam_stis'],simdata['flux_obs'])

ax1.set_title('STIS E140H spectrum, S/N=25',size=25)
ax1.set_xlabel(r'$\lambda_{\rm obs} (\AA)$',size=25)
ax1.set_ylabel('flux',size=20)
ax1.annotate(fo['SyntheticData'].header['LINE'],(0.6,0.18),xycoords='axes fraction',size=25)
ax1.annotate(r'$z_{\rm obs}$'+'={:4.2f}'.format(fo[3].header['OBS_Z']),(0.6,0.08),xycoords='axes fraction',size=25)
ax2.set_title('Pristine input spectrum',size=25)
ax2.set_xlabel(r'$\lambda_{\rm obs} (\AA)$',size=25)
ax2.set_ylabel('flux',size=20)
ax2.annotate(fo['SyntheticData'].header['LINE'],(0.6,0.15),xycoords='axes fraction',size=25)
ax2.annotate(r'$z_{\rm obs}$'+'={:4.2f}'.format(fo[3].header['OBS_Z']),(0.6,0.08),xycoords='axes fraction',size=25)

fig2.savefig(os.path.join(save_location,'asdi2.png'),dpi=300)

In [ ]:
#define a helper function for looping over parameters
def run_stis_datasim(line_name,ext_num,path,snratioresel=25,override=False,save_location='.'):

    sn_string='-sn{:d}'.format(snratioresel)
    
    STIS_sim_filename = os.path.basename(example_path).replace('pristine', 'stis'+sn_string)
    
    stis_file_path=os.path.join(save_location,STIS_sim_filename)
    
    inst_sim_url = "https://masttest.stsci.edu/asdi/api/v0.1/addsignature"
    PARAMS = {'instrument':'STIS', 
              'line_name': line_name, 
              'ext_num': ext_num, 
              'public_path': path,
              'snratioresel': snratioresel,
              'override_observed_wavelength': override}
    
    #skip if this file already exists locally
    if os.path.lexists(stis_file_path):
        print('simulated file exists, skipping service call.')
        return stis_file_path
    
    r = requests.get(url = inst_sim_url, params = PARAMS, allow_redirects = True) 

    if r.status_code != 200: #show what went wrong
        #print(r.headers)
        print(r.text)
    else:
        open(stis_file_path, 'wb').write(r.content)

    return stis_file_path

In [ ]:
# Simulate STIS data with several Signal-to-Noise ratios
sn_list=[10,20,100]

fig3,ax1 = pyplot.subplots(1,1, figsize=(10, 6),dpi=72)
pyplot.subplots_adjust(hspace=0.2,wspace=0.2,top=0.92,bottom=0.12,left=0.10,right=0.99)

#recall that the "example_" parameters were for our selected LOS above.
for sn in sn_list:
    filepath=run_stis_datasim(example_line_name,example_ext_num,example_path,
                              snratioresel=sn,
                              override=1500.0,
                              save_location=save_location)
    print(filepath)
    #open each file successively and overplot
    this_fo=fits.open(filepath)
    this_simdata=this_fo['SyntheticData'].data
    flux=this_simdata['flux_stis']
    ax1.plot(this_simdata['lam_stis'],flux,alpha=(sn/100.0)**0.33)

ax1.set_title('STIS E140H spectra, S/N=10,20,100',size=22)
ax1.set_xlabel(r'Observed Wavelength $(\AA)$',size=22)
ax1.set_ylabel('flux',size=20)
ax1.annotate(fo['SyntheticData'].header['LINE'],(0.6,0.18),xycoords='axes fraction',size=22)
ax1.annotate(r'$z_{\rm obs}$'+'={:4.2f}'.format(this_fo[3].header['OBS_Z']),(0.6,0.08),xycoords='axes fraction',size=22)

fig3.savefig(os.path.join(save_location,'asdi3.png'),dpi=300)

### Contributors
* Theresa Dower
* Gregory Snyder
* Molly Peeples